In [101]:
import pandas as pd
import numpy as np
import csv
%matplotlib inline
%config InlineBackend.figure_format='retina'
# import libraries
import math
import numpy as np
import matplotlib as mp
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
# from importlib import reload
from datetime import datetime
from IPython.display import Image
from IPython.display import display_html
from IPython.display import display
from IPython.display import Math
from IPython.display import Latex
from IPython.display import HTML
from sklearn.metrics import mean_squared_error
import scipy as sp
import scipy.sparse.linalg as linalg
import scipy.cluster.hierarchy as hr
from scipy.spatial.distance import pdist, squareform

import sklearn.datasets as datasets
import sklearn.metrics as metrics
import sklearn.utils as utils
import sklearn.linear_model as linear_model
import sklearn.model_selection as model_selection
import sklearn.cluster as cluster
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm

from patsy import dmatrices

import seaborn as sns

print('')

In [2]:
from sklearn.feature_extraction import DictVectorizer

In [159]:
df = pd.read_csv("airbnb1.csv")
df = df.drop('latitude', axis=1)
df = df.drop('longitude', axis=1)
df = df.drop('price_class', axis= 1)
# df = df.drop('neighborhood', axis=1)
df = df.drop('id', axis=1)
df2 = pd.read_csv("airbnb_cnt.csv")

df_ref = df

In [160]:
reference_list = ['Allston-Brighton','Dorchester',
                  'Back Bay', 'South Boston',
                  'Brookline','Downtown', 
                  'East Boston', 'Fenway/Kenmore',
                  'Jamaica Plain','East Boston','Beacon Hill',]
df = df[(df.neighborhood == 'Allston-Brighton') | (df.neighborhood == 'Dorchester')| 
        (df.neighborhood == 'Back Bay') | (df.neighborhood == 'South Boston')|
       (df.neighborhood == 'Brookline') | (df.neighborhood == 'Downtown')|
       (df.neighborhood == 'East Boston') | (df.neighborhood == 'Fenway/Kenmore')|
       (df.neighborhood == 'Jamaica Plain') | (df.neighborhood == 'East Boston') |
       (df.neighborhood == 'Beacon Hill') ]
neighbor_list = []
for x in df['neighborhood']:
    dic = dict()
    dic['neighborhood'] = x
    neighbor_list.append(dic)
vec = DictVectorizer()
X = vec.fit_transform(neighbor_list)
df_neighbor = pd.DataFrame(X.toarray(), columns = vec.get_feature_names())
df = df.drop('neighborhood', axis = 1)
df_ori = df
df = df.reset_index(drop=True)
df = pd.concat([df_neighbor,df], axis = 1)
# print(df_neighbor.shape)
# df_neighbor = pd.concat([df_neighbor,df['bathrooms/bed']], axis = 1)
print(df.shape)

(3075, 15)


In [161]:
df['restaurant_count'] =df2['restaurant_count']
df['conveniencet_count'] = df2['conveniencet_count']
df['subway_count'] = df2['subway_count']
df['id'] = df2['id']
df_ori['restaurant_count'] =df2['restaurant_count']
df_ori['conveniencet_count'] = df2['conveniencet_count']
df_ori['subway_count'] = df2['subway_count']
df_ori['id'] = df2['id']
df

,neighborhood=Allston-Brighton,neighborhood=Back Bay,neighborhood=Beacon Hill,neighborhood=Brookline,neighborhood=Dorchester,neighborhood=Downtown,neighborhood=East Boston,neighborhood=Fenway/Kenmore,neighborhood=Jamaica Plain,neighborhood=South Boston,bathrooms/bed,reviewscore_rating,score_cleanliness,score_location,price,restaurant_count,conveniencet_count,subway_count,id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.000000,99.0,10.0,10.0,95.00,4,1,0,20872145
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.000000,93.0,9.0,9.0,55.00,5,2,0,13937181
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.000000,100.0,10.0,10.0,95.00,4,2,0,3575220
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.000000,98.0,10.0,9.0,58.00,0,0,0,17419858
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.000000,0.0,0.0,0.0,60.00,1,0,0,18202496
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.000000,96.0,10.0,9.0,40.00,2,1,0,16987125
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.500000,99.0,10.0,9.0,75.00,4,1,0,1668313
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.500000,99.0,10.0,10.0,60.00,5,1,0,13768853
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.000000,100.0,10.0,10.0,67.00,2,1,0,19248168
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.000000,0.0,0.0,0.0,51.00,0,0,0,849408


In [162]:
new_column_names = [x.replace(' ','_') for x in df.columns]
new_column_names = [x.replace('=','_') for x in df.columns]
df.columns = new_column_names
print(df.columns)

Index(['neighborhood_Allston-Brighton', 'neighborhood_Back Bay',
       'neighborhood_Beacon Hill', 'neighborhood_Brookline',
       'neighborhood_Dorchester', 'neighborhood_Downtown',
       'neighborhood_East Boston', 'neighborhood_Fenway/Kenmore',
       'neighborhood_Jamaica Plain', 'neighborhood_South Boston',
       'bathrooms/bed', 'reviewscore_rating', 'score_cleanliness',
       'score_location', 'price', 'restaurant_count', 'conveniencet_count',
       'subway_count', 'id'],
      dtype='object')


In [163]:
new_column_names = [x.replace(' ','_') for x in df_ori.columns]
new_column_names = [x.replace('=','_') for x in df_ori.columns]
df_ori.columns = new_column_names
print(df_ori.columns)

Index(['bathrooms/bed', 'reviewscore_rating', 'score_cleanliness',
       'score_location', 'price', 'restaurant_count', 'conveniencet_count',
       'subway_count', 'id'],
      dtype='object')


In [164]:
df.fillna(0, inplace=True)
print(df.shape)

(3075, 19)


In [165]:
df_ori.fillna(0, inplace=True)
print(df_ori.shape)

(3075, 9)


In [167]:
X = df[df.columns.difference(['price'])]
X = X[X.columns.difference(['id'])]
y = df["price"]

In [168]:
df.columns

Index(['neighborhood_Allston-Brighton', 'neighborhood_Back Bay',
       'neighborhood_Beacon Hill', 'neighborhood_Brookline',
       'neighborhood_Dorchester', 'neighborhood_Downtown',
       'neighborhood_East Boston', 'neighborhood_Fenway/Kenmore',
       'neighborhood_Jamaica Plain', 'neighborhood_South Boston',
       'bathrooms/bed', 'reviewscore_rating', 'score_cleanliness',
       'score_location', 'price', 'restaurant_count', 'conveniencet_count',
       'subway_count', 'id'],
      dtype='object')

In [169]:
X_ori = df_ori[df_ori.columns.difference(['price'])]
X_ori = X_ori[X_ori.columns.difference(['id'])]
y_ori = df_ori["price"]

Cause the price is maily dominated by the neighborhood, in order to analyse the influence of some features, we use the model without neighborhood, and compare the square root mean square error.

In [170]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle = True)
neighbor_in_sum = 0
neighbor_out_sum = 0
flag = 0
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    X_train_ori, X_test_ori = X_ori.iloc[train_index], X_ori.iloc[test_index]
    y_train_ori, y_test_ori = y_ori.iloc[train_index], y_ori.iloc[test_index]
    result = sm.OLS(y_train, X_train).fit()
    if flag == 0:
        print(result.summary())
        flag += 1
    result_ori = sm.OLS(y_train_ori, X_train_ori).fit()
    if flag == 1:
        print(result_ori.summary())
        flag += 1
    y_pred = result.predict(X_test)
    y_pred_ori = result_ori.predict(X_test_ori)
    y_pred = result.predict(X_test)
    e_in = math.sqrt(mean_squared_error(y_test, y_pred))
    e_out = math.sqrt(mean_squared_error(y_test_ori, y_pred_ori))
    neighbor_in_sum += e_in
    neighbor_out_sum += e_out
    neighbor_balance = (e_in + e_out) / 2
    print("Error of neibourhood in model is {}".format(e_in))
    print("Error of neibourhood out model is {}".format(e_out))
print("Average error of neibourhood in model is {}".format(neighbor_in_sum / 5))
print("Average error of neibourhood out model is {}".format(neighbor_out_sum / 5))

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.239
Model:                            OLS   Adj. R-squared:                  0.234
Method:                 Least Squares   F-statistic:                     48.07
Date:                Wed, 02 May 2018   Prob (F-statistic):          2.40e-132
Time:                        18:06:26   Log-Likelihood:                -14138.
No. Observations:                2460   AIC:                         2.831e+04
Df Residuals:                    2443   BIC:                         2.841e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
bathrooms/bed 

In [172]:
X_ori = X_ori[X_ori.columns.difference(['reviewscore_rating'])]
X_ori = X_ori[X_ori.columns.difference(['score_location'])]
X_ori = X_ori[X_ori.columns.difference(['subway_count'])]
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle = True)
neighbor_in_sum = 0
neighbor_out_sum = 0
flag = 0
for train_index, test_index in kf.split(X):
    X_train_ori, X_test_ori = X_ori.iloc[train_index], X_ori.iloc[test_index]
    y_train_ori, y_test_ori = y_ori.iloc[train_index], y_ori.iloc[test_index]
    result_ori = sm.OLS(y_train_ori, X_train_ori).fit()
    if flag == 0:
        print(result_ori.summary())
        flag += 1
    y_pred_ori = result_ori.predict(X_test_ori)
    e_out = math.sqrt(mean_squared_error(y_test_ori, y_pred_ori))
    neighbor_out_sum += e_out

    print("Error of neibourhood out model is {}".format(e_out))
print("Average error of neibourhood out model is {}".format(neighbor_out_sum / 5))

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.599
Model:                            OLS   Adj. R-squared:                  0.599
Method:                 Least Squares   F-statistic:                     918.2
Date:                Wed, 02 May 2018   Prob (F-statistic):               0.00
Time:                        18:06:34   Log-Likelihood:                -14554.
No. Observations:                2460   AIC:                         2.912e+04
Df Residuals:                    2456   BIC:                         2.914e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
bathrooms/bed         46.9211      3

We can see from the result that, the error does not change much when we remove 'reviewscore_rating', 'score_location' and 'subway_count'. So we can keep that out of our model.

# Recommendation part

In [ ]:
df = df.drop('reviewscore_rating', 1)
df = df.drop('score_location', 1)
df = df.drop('subway_count', 1)

In [184]:

X = df[df.columns.difference(['price'])]
X = X[X.columns.difference(['id'])]
y = df["price"]
result = sm.OLS(y, X).fit()

In [182]:
df_selct = df[(df.neighborhood_Downtown == 1)]
id_selct = df_selct['id']
y_selct = df_selct['price']
df_selct.columns
X_selct= df_selct[df_selct.columns.difference(['price'])]
X_selct = X_selct[X_selct.columns.difference(['id'])]
print(id_selct.shape)

(23,)


In [185]:
y_pred_selct = result.predict(X_selct)

In [216]:
y_dif = y_selct - y_pred_selct
y_dif = y_dif.to_frame(name = 'difference')
y_reco = y_dif[y_dif.difference < 0]
reco_list = y_reco.index
reco_list

Int64Index([1171, 1172, 1178, 1245, 1379, 1380, 1381, 1382, 1383, 1393, 1396,
            1398, 1400, 1408, 1416, 1417],
           dtype='int64')

//y_dif > 0 means the ture price is greater than price we predicted. What we will recommand to customers is those true price lower than predied, whose y_dif < 0.

In [ ]:
id_selct = id_selct.to_frame(name = 'id')


In [227]:
reco_id = []
for index, row in id_selct.iterrows():
    if index in reco_list:
        reco_id.append(row['id'])

In [229]:
print(reco_id)
print(len(reco_id))

[10787333, 13918656, 20960666, 8815738, 7694559, 15609432, 2495047, 20137739, 5074924, 7294565, 18213171, 1200502, 20085832, 21047722, 798957, 9430774]
16
